In [1]:
%pylab
from glob import glob
from natsort import natsorted
import h5py
from scipy.stats import binned_statistic
import palettable.colorbrewer.qualitative
from matplotlib.ticker import ScalarFormatter, NullFormatter,FormatStrFormatter

from palettable.colorbrewer.qualitative import Dark2_7
#from palettable.cubehelix import Cubehelix
#palette = Cubehelix.make(n=10)

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
 natsorted(glob("stirring/*"))

In [3]:
plt.clf()
fig, ax = plt.subplots(2,4,figsize=(8,4))
fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/*/stars"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e4

labeldict = {"sinkdata/b_default_n_n/stars": r"TURBSPHERE",
             "sinkdata/b_default_y_n/stars": r"TURBSPHERE+Driving",
             "sinkdata/b_default_n_y/stars": r"TURBSPHERE+Jets",
             "sinkdata/b_default_y_y/stars": r"TURBSPHERE+Driving+Jets",
             "sinkdata/box/stars": r"BOX+Driving",
             "sinkdata/box_jets/stars": r"BOX+Driving+Jets",
             "sinkdata/sphere/stars": r"SPHERE",
             "sinkdata/sphere_jets/stars": r"SPHERE+Jets",
            }

plt.rcParams.update({'axes.labelsize': 6})
#plt.rcParams.update({'axes.ticklabelsize': 8})

plt.rc('xtick',labelsize=6)
plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d
    if not d in labeldict.keys(): continue
#    if not "fw0.3_fk0.3" in d: continue
#    if not ("THETA" in d or "ISOTROPIC" in d): continue
    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue
        #files = natsorted(glob(d+"/output_old/snapshot*stars.hdf5"))

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    
#    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
#    dm = dm.replace("-0","{-}")
    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if "PartType5" in F.keys(): 
                mstar = np.array(F["PartType5"]["Masses"])
    #            if not (mstar>0.08).sum(): continue
                mstar = mstar[mstar>1e-2]
            else:
                mstar = np.zeros(1)
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())

            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            machs.append(np.array(F["Mach"]))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    print(mtot.max())
    #ls = ('solid' if i < 7 else 'dashed')
    if "b_default" in d: ls='solid'
    elif "box" in d: ls='dashed'
    else: ls='dashdot'
        
    times -= times[Nsink>1][0]
    ax[0,0].plot(times, mtot/M0, label=labeldict[name], color=color,ls=ls,zorder=z,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
#    ax[0,0].plot([-1,-1],label="Kroupa IMF")
    ax[0,1].plot(times, Nsink,lw=1,ls=ls,zorder=z,color=color)
    ax[0,2].plot(mtot/M0, mquantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
#    ax[0,2].plot([0,1000],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(mtot/M0, quantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
#    ax[1,0].plot([0,1000],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(mtot/M0, means,lw=1,ls=ls,zorder=z,color=color)
#    ax[1,1].plot([0,1000],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,3].plot(mtot/M0, quantiles[:,-1],lw=1,ls=ls,zorder=z,color=color)
    ax[1,0].plot(times, machs,zorder=z,lw=1,ls=ls,color=color)
    i += 1
Tmax=12
ax[0,0].set(yscale='log',xlim=[0.1,Tmax],ylim=[3e-4,0.3], ylabel="SFE",xlabel=r"$t$ (Myr)",xscale='log')
#ax[0,0].set_xlabel("Time(Myr)", fontsize=8)
ax[0,1].set(yscale='log',xscale='log',ylim=[1,8000], xlim=[0.1,Tmax],ylabel="$N_\mathrm{*}$",xlabel=r"$t$ (Myr)")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.8,130], xlim=[1e-3,0.3], ylabel="$M_\mathrm{50}\,(M_\odot)$",xlabel=r"SFE")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.03,10], xlim=[1e-3,0.3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel=r"SFE")
ax[1,2].set(yscale='log',xscale='log',ylim=[0.2,30], xlim=[1e-3,0.3],ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel=r"SFE")
ax[1,3].set(yscale='log',xscale='log',ylim=[1,1000], xlim=[1e-3,0.3], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel=r"SFE")
ax[1,0].set(yscale='log',xscale='log', xlim=[1,Tmax], ylim=[7,30],xlabel=r"$t$ (Myr)",ylabel="$\mathcal{M}$")
#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


#f = FormatStrFormatter("%g")
#f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
#ax[1,0].xaxis.set_major_formatter(f)
#ax[,0].xaxis.set_minor_formatter(f)
#for a in ax.flatten()[1:]:
#    a.yaxis.set_major_formatter(f)
#    a.xaxis.set_major_formatter(f)


ax[0,0].legend(labelspacing=0,fontsize=7,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

fig.subplots_adjust(hspace=0.3,wspace=0.35)
plt.savefig("M2e4_Turbsphere_SFE_IMF.pdf",bbox_inches='tight')

sinkdata/b_default_n_n/stars


/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log10
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in true_divide
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less


4338.69921875
sinkdata/b_default_n_y/stars


/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log10
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in true_divide
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less


2200.806396484375
sinkdata/b_default_y_n/stars


...

294.4752197265625
sinkdata/sphere/stars


/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log10
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in true_divide
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less


5843.93310546875
sinkdata/sphere_jets/stars


/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log10
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in true_divide
/home/mike/.local/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less


2137.0234375


In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax.

In [8]:
mtot/M0

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        print(f)
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue                
            mstar = np.array(F["PartType5"]["BH_Mass"])
            mstar = mstar[np.array(F["PartType5"]["BH_Mass"])/np.array(F["PartType5"]["BH_Mdot"])*1e9 > 1e8]
            if len(mstar)<2: continue
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


#f = FormatStrFormatter("%g")
#f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
#ax[1,0].xaxis.set_major_formatter(f)
#ax[1,0].xaxis.set_minor_formatter(f)
#for a in ax.flatten():
#    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_MassResolution_NonAccreting.pdf",bbox_inches='tight')
    

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence_heating/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue
            mstar = np.array(F["PartType5"]["BH_Mass"])
#            if not (mstar>0.08).sum(): continue
#            mstar = mstar[mstar>0.08]
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    if len(mquantiles) == 0: continue
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
ax[1,0].xaxis.set_major_formatter(f)
ax[1,0].xaxis.set_minor_formatter(f)
for a in ax.flatten():
    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_Heating_MassResolution.pdf",bbox_inches='tight')
    

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence_heating/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue
            mstar = np.array(F["PartType5"]["BH_Mass"])
            if not (mstar>0.08).sum(): continue
            mstar = mstar[mstar>0.08]
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[2,2],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.23,0.23],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.6,0.6],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
ax[1,0].xaxis.set_major_formatter(f)
ax[1,0].xaxis.set_minor_formatter(f)
for a in ax.flatten():
    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_MassResolution_Heating_NoBrownDwarfs.pdf",bbox_inches='tight')